In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Path to working directory", "dataset_name") # Esto es para descartar el dataset inicial, poner el path en tu ordenador local

print("Path to dataset files:", path)

Path to dataset files: C:\Users\mique\.cache\kagglehub\datasets\tarkkaanko\amazon\versions\1


In [10]:
import pandas as pd
df = pd.read_csv('amazon_reviews.csv')
df.head() #Aqui podemos ver el dataset usado, varias columnas innecesarias

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [11]:
#Limpiar el dataset
dfSimple = df.drop(columns=['reviewerName', 'reviewTime', 'day_diff', 'helpful_yes', 'helpful_no', 'total_vote', 'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'])
dfSimple.head()
#De inicio trabajaremos con un dataset masivamente simplificado para poder hacer pruebas, la columna unnamed:0 no se elimina porque es un identificador de la fila y puede ser util mas adelante


,Unnamed: 0,overall,reviewText
0,0,4.0,No issues.
1,1,5.0,"Purchased this for my device, it worked as adv..."
2,2,4.0,it works as expected. I should have sprung for...
3,3,5.0,This think has worked out great.Had a diff. br...
4,4,5.0,"Bought it with Retail Packaging, arrived legit..."


In [13]:
#Por ahora, trabajaremos con valores de 0 o 1 en la columna overall para tener un problema simple y poder de inicio identifcar de manera binaria si es positiva o negativa la review. Despues, querremos predecir la nota de la review por el texto con mas exactitud
dfSimpleBinary = dfSimple.copy()
dfSimpleBinary['overall'] = dfSimpleBinary['overall'].apply(lambda x: 1 if x > 2.5 else 0)
dfSimpleBinary['overall'].value_counts()
dfSimpleBinary.head(1000)

,Unnamed: 0,overall,reviewText
0,0,1,No issues.
1,1,1,"Purchased this for my device, it worked as adv..."
2,2,1,it works as expected. I should have sprung for...
3,3,1,This think has worked out great.Had a diff. br...
4,4,1,"Bought it with Retail Packaging, arrived legit..."
...,...,...,...
995,995,1,Gave my galaxy S4 plenty of memory and at a go...
996,996,1,Bought a digital recorder with a memory card s...
997,997,1,"Not much to say, its a memory card but it work..."
998,998,1,32 gig MicroSD card that costs less than a dol...


In [16]:
#Guardamos el dataset limpio para no sobreescribirlo accidentalmente, este paso es solo una precaucion. solo falta borrar los nans
dfSimpleBinary.dropna(subset=["reviewText"], inplace=True)
dfSimpleBinary.to_csv('amazon_reviews_simpleBinary.csv', index=False)